In [1]:
import numpy as np
import pandas as pd
import os
import json
import cv2
from tqdm import tqdm
import zipfile
import torch
from ultralytics import YOLO
import time
import shutil


import zipfile

DIR = r'data\SoccerNetGS\gamestate-2024'


def unzip_data():
    paths = ['challenge.zip', 'test.zip', 'train.zip', 'valid.zip']
    for path in paths:
        f_path = os.path.join(DIR, path)
        with zipfile.ZipFile(f_path, 'r') as zip_ref:
            zip_ref.extractall(os.path.join(DIR, path[:-4]))

#unzip_data()  

In [54]:
DIRECTORY = 'data/SoccerNetGS/gamestate-2024/train/'


def load_data(dir: str = None) -> list:
    data = list()
    print('Wczytywanie danych...')
    for folder_name in tqdm(os.listdir(DIRECTORY)):
        folder_full_path = os.path.join(DIRECTORY, folder_name)
        if os.path.isdir(folder_full_path):
            with open(os.path.join(DIRECTORY, folder_name, 'Labels-GameState.json')) as f:
                d = json.load(f)
                data.append(d['annotations'])
    return data


def parse_data_to_df(data: list) -> pd.DataFrame:
    start = time.time()
    print('Tworzenie DataFrame...')
    flat_dict = dict()
    for annotations in tqdm(data):
        data_image = filter(lambda x: x['supercategory'] == 'object' 
                                      and x['bbox_pitch_raw'] is not None, annotations)
        for img in data_image:
            for key in img:
                if isinstance(img[key], dict):
                    for key2 in img[key]:
                        if key + '_' + key2 not in flat_dict:
                            flat_dict[key + '_' + key2] = []
                        flat_dict[key + '_' + key2].append(img[key][key2])
                else:
                    if key not in flat_dict:
                        flat_dict[key] = []
                    flat_dict[key].append(img[key])
    df = pd.DataFrame(flat_dict, index=flat_dict['id'])
    df = df.drop(columns=['id'])
    return df


In [55]:
data = load_data()

Wczytywanie danych...


100%|██████████| 58/58 [00:17<00:00,  3.41it/s]


In [56]:
df = parse_data_to_df(data)

Tworzenie DataFrame...


100%|██████████| 57/57 [00:04<00:00, 11.80it/s]


In [ ]:
for item in os.listdir(DIRECTORY):
    print(item[-3:])

In [ ]:
with open('data/SoccerNetGS/gamestate-2024/valid/SNGS-021/img1/000001.jpg') as f:
    img = cv2.imread(f)
    print(img)

In [59]:
for dir in os.listdir(DIRECTORY):
    if os.path.isdir(os.path.join(DIRECTORY, dir)):
        img1_path = os.path.join(DIRECTORY, dir, 'img1')
        for img in os.listdir(img1_path):
            shutil.copy(os.path.join(img1_path, img), f'datasets/football_data/train/images/{dir[-3:]}{img}')

In [5]:
df

,image_id,track_id,supercategory,category_id,attributes_role,attributes_jersey,attributes_team,bbox_image_x,bbox_image_y,bbox_image_x_center,...,bbox_pitch_x_bottom_right,bbox_pitch_y_bottom_right,bbox_pitch_x_bottom_middle,bbox_pitch_y_bottom_middle,bbox_pitch_raw_x_bottom_left,bbox_pitch_raw_y_bottom_left,bbox_pitch_raw_x_bottom_right,bbox_pitch_raw_y_bottom_right,bbox_pitch_raw_x_bottom_middle,bbox_pitch_raw_y_bottom_middle
2021000001,2021000001,1,object,1,player,3,right,1699,236,1715.5,...,15.432421,-20.763304,15.098004,-20.797046,14.856742,-20.818659,15.540940,-20.751110,15.199319,-20.785023
2021000002,2021000001,2,object,1,player,8,right,1658,408,1683.5,...,10.062544,2.378321,9.704683,2.368070,9.474327,2.191314,10.206753,2.211401,9.841066,2.201362
2021000003,2021000001,3,object,1,player,14,right,1004,925,1040.0,...,0.419694,24.264031,0.123195,24.263700,-0.169058,24.302677,0.417822,24.303411,0.124492,24.303362
2021000004,2021000001,4,object,1,player,28,right,1034,682,1069.0,...,0.819709,18.209514,0.468589,18.207035,0.121098,18.252899,0.823548,18.257540,0.472493,18.255491
2021000005,2021000001,5,object,1,player,7,right,994,404,1012.0,...,0.202874,1.338812,-0.072267,1.331025,-0.367440,1.318310,0.182895,1.332912,-0.092189,1.325603
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2096017811,2096000750,21,object,1,player,8,left,1591,516,1621.5,...,34.675215,30.218365,34.209444,29.829257,34.215451,29.194610,34.888597,29.768208,34.552373,29.481815
2096017812,2096000750,22,object,1,player,8,right,1441,448,1469.0,...,37.701615,24.647054,37.355894,24.351442,37.020587,24.040611,37.719071,24.647982,37.370185,24.344579
2096017813,2096000750,23,object,1,player,50,right,1276,616,1323.5,...,26.160493,31.406246,25.831511,31.130727,25.884768,30.664757,26.797504,31.438215,26.341256,31.052171
2096017814,2096000750,24,object,2,goalkeeper,None,right,1099,234,1112.0,...,49.025133,4.171888,48.791924,3.970881,48.680078,3.573496,49.157395,3.995274,48.918879,3.784316


In [24]:
df[["attributes_role", "attributes_team"]]

,attributes_role,attributes_team
2021000001,player,right
2021000002,player,right
2021000003,player,right
2021000004,player,right
2021000005,player,right
...,...,...
2096017811,player,left
2096017812,player,right
2096017813,player,right
2096017814,goalkeeper,right


In [ ]:
def draw_bbox(image, bbox, color=(0, 0, 0)):
    x, y, w, h = bbox
    cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
    img_height, img_width, _ = image.shape
    scale_factor = min(1000 / img_width, 720 / img_height)
    if scale_factor < 1:
        img = cv2.resize(image, (int(img_width * scale_factor), int(img_height * scale_factor)))
    cv2.imshow('Bounding Box', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return img

In [ ]:
image = cv2.imread('data/SoccerNetGS/gamestate-2024/train/SNGS-060/img1/000001.jpg')
bbox = df.loc['2021000001', ['bbox_image_x', 'bbox_image_y', 'bbox_image_w', 'bbox_image_h']]
bbox = bbox.values
img = draw_bbox(image, bbox)

In [39]:
def get_parsed_df(path: str):
    data = list()
    with open(os.path.join(path, 'Labels-GameState.json')) as f:
        d = json.load(f)
        data.append(d['annotations'])
    flat_dict = dict()
    for annotations in data:
        data_annotations = filter(lambda x: x['supercategory'] == 'object', annotations)
        for img in data_annotations:
            if img['bbox_pitch_raw'] is None:
                # set bbox_pitch to 0,0,0,0
                img['bbox_pitch_raw'] = {"x_bottom_left": -0.5618667663373333,
                                     "y_bottom_left": 24.046080743490077,
                                     "x_bottom_right": 0.0073212402595505275,
                                     "y_bottom_right": 24.050080078959525,
                                     "x_bottom_middle": -0.2772623829267213,
                                     "y_bottom_middle": 24.048314011381127}
            for key in img:
                if isinstance(img[key], dict):
                    for key2 in img[key]:
                        if key + '_' + key2 not in flat_dict:
                            flat_dict[key + '_' + key2] = []
                        flat_dict[key + '_' + key2].append(img[key][key2])
                else:
                    if key not in flat_dict:
                        flat_dict[key] = []
                    flat_dict[key].append(img[key])
    df = pd.DataFrame(flat_dict, index=flat_dict['id'])
    df = df.drop(columns=['id'])
    return df

In [40]:
df_parsed = get_parsed_df('data/SoccerNetGS/gamestate-2024/train/SNGS-060/')
df_parsed

ValueError: Length of values (8668) does not match length of index (8677)

In [34]:
df[df['image_id'].isin(['2021000001', '2021000002'])]

,image_id,track_id,supercategory,category_id,attributes_role,attributes_jersey,attributes_team,bbox_image_x,bbox_image_y,bbox_image_x_center,...,bbox_pitch_x_bottom_right,bbox_pitch_y_bottom_right,bbox_pitch_x_bottom_middle,bbox_pitch_y_bottom_middle,bbox_pitch_raw_x_bottom_left,bbox_pitch_raw_y_bottom_left,bbox_pitch_raw_x_bottom_right,bbox_pitch_raw_y_bottom_right,bbox_pitch_raw_x_bottom_middle,bbox_pitch_raw_y_bottom_middle
2021000001,2021000001,1,object,1,player,3,right,1699,236,1715.5,...,15.432421,-20.763304,15.098004,-20.797046,14.856742,-20.818659,15.540940,-20.751110,15.199319,-20.785023
2021000002,2021000001,2,object,1,player,8,right,1658,408,1683.5,...,10.062544,2.378321,9.704683,2.368070,9.474327,2.191314,10.206753,2.211401,9.841066,2.201362
2021000003,2021000001,3,object,1,player,14,right,1004,925,1040.0,...,0.419694,24.264031,0.123195,24.263700,-0.169058,24.302677,0.417822,24.303411,0.124492,24.303362
2021000004,2021000001,4,object,1,player,28,right,1034,682,1069.0,...,0.819709,18.209514,0.468589,18.207035,0.121098,18.252899,0.823548,18.257540,0.472493,18.255491
2021000005,2021000001,5,object,1,player,7,right,994,404,1012.0,...,0.202874,1.338812,-0.072267,1.331025,-0.367440,1.318310,0.182895,1.332912,-0.092189,1.325603
2021000006,2021000001,6,object,1,player,None,right,971,385,994.5,...,0.019875,-0.344371,-0.317726,-0.354470,-0.739954,-0.370923,0.003264,-0.350034,-0.368204,-0.360514
2021000007,2021000001,7,object,1,player,None,right,1015,190,1028.5,...,0.685412,-31.858087,0.341208,-31.880788,-0.004754,-31.934621,0.683062,-31.891696,0.339289,-31.913380
2021000008,2021000001,8,object,1,player,None,right,1580,342,1604.0,...,10.298898,-4.489486,9.905286,-4.508260,9.511378,-4.566998,10.299780,-4.530941,9.906139,-4.549041
2021000009,2021000001,9,object,3,referee,None,None,1460,619,1485.5,...,5.184011,16.051417,4.899362,16.052855,4.645077,16.243796,5.178156,16.240247,4.911838,16.242143
2021000010,2021000001,10,object,1,player,13,left,762,492,783.5,...,-2.703111,8.933945,-2.997441,8.926923,-3.330795,8.962259,-2.773610,8.974899,-3.052192,8.968632


In [25]:
df[df['image_id'].str.startswith('033000076')]

,image_id,track_id,supercategory,category_id,attributes_role,attributes_jersey,attributes_team,bbox_image_x,bbox_image_y,bbox_image_x_center,...,bbox_pitch_x_bottom_right,bbox_pitch_y_bottom_right,bbox_pitch_x_bottom_middle,bbox_pitch_y_bottom_middle,bbox_pitch_raw_x_bottom_left,bbox_pitch_raw_y_bottom_left,bbox_pitch_raw_x_bottom_right,bbox_pitch_raw_y_bottom_right,bbox_pitch_raw_x_bottom_middle,bbox_pitch_raw_y_bottom_middle


In [57]:
def get_YOLO_file(df: pd.DataFrame, split: str):
    categories = {'ball': 1, 'player': 0, 'referee': 3, 'goalkeeper': 2}
    df = df.loc[:, ['image_id', 'bbox_image_x_center', 'bbox_image_y_center', 'bbox_image_w', 'bbox_image_h', 'attributes_role']]
    df = df[df['attributes_role'].isin(categories.keys())]
    df_dict = {image_id: df_group for image_id, df_group in df.groupby('image_id')}
    #file_contents = ['' for _ in range(df.image_id.nunique())]
    for image_id, df_img in tqdm(df_dict.items()):
        file_content = []
        for i, row in df_img.iterrows():
            x, y, w, h = row[['bbox_image_x_center', 'bbox_image_y_center', 'bbox_image_w', 'bbox_image_h']]
            role = row['attributes_role']
            category = categories[role]
            x = round(x / 1920, 6)
            y = round(y / 1080, 6)
            w = round(w / 1920, 6)
            h = round(h / 1080, 6)
            file_content.append(f'{category} {x} {y} {w} {h}\n')
        with open(f'datasets/football_data/{split}/labels/{image_id[1:]}.txt', 'w') as f:
            f.writelines(file_content)

In [60]:
get_YOLO_file(df, 'train')

100%|██████████| 41627/41627 [02:53<00:00, 239.95it/s]


In [61]:
labels_files = os.listdir('datasets/football_data/train/labels')
image_files = os.listdir('datasets/football_data/train/images')

In [62]:
labels_files_mapped = list(map(lambda x: x[:-4], labels_files))
image_files_mapped = list(map(lambda x: x[:-4], image_files))

In [63]:
not_matched_images = list(set(image_files_mapped) - set(labels_files_mapped))

In [64]:
not_matched_images_mapped = list(map(lambda x: x + '.jpg', not_matched_images))
not_matched_images_mapped = sorted(not_matched_images_mapped)

In [67]:
for image in not_matched_images_mapped:
    os.remove(f'datasets/football_data/train/images/{image}')

In [68]:
len(os.listdir('datasets/football_data/valid/images')) == len(os.listdir('datasets/football_data/valid/labels'))

True

In [ ]:
get_YOLO_file('data/SoccerNetGS/gamestate-2024/train/SNGS-060/', 'train')

In [ ]:
get_YOLO_file('data/SoccerNetGS/gamestate-2024/valid/SNGS-021/', 'valid')

In [ ]:
def draw_bbox_from_yolo_file(path: str, img_num: int):
    filename = f'{img_num:06d}'
    img = cv2.imread(f'{path}/images/{filename}.jpg')
    with open(f'{path}/labels/{filename}.txt', 'r') as f:
        labels = f.read()
    print(labels)
draw_bbox_from_yolo_file('datasets/football_data/train', 1)

In [ ]:
path = 'data/SoccerNetGS/gamestate-2024/valid/SNGS-021/Labels-GameState.json'
with open(path) as f:
    loaded_data = json.load(f)
loaded_data = loaded_data['annotations']
loaded_data

In [ ]:
def flatten_data(data: list[list[dict]]) -> list:
    flat_list = [y for x in data for y in x]
    return flat_list

In [ ]:
flatten_data_list: list[dict] = flatten_data(data)

In [ ]:
def get_pitch_data_df(data: list) -> pd.DataFrame:
    pitch_data_dict = {'id': [], 'image_id': [], 'line_type': [], 'x': [], 'y': []}
    data = filter(lambda x: x['supercategory'] == 'pitch', data)
    for anotation in tqdm(data):
        for line in anotation['lines']:
            for data_point in anotation['lines'][line]:
                pitch_data_dict['id'].append(int(anotation['id']))
                pitch_data_dict['image_id'].append(int(anotation['image_id']))
                pitch_data_dict['line_type'].append(line)
                pitch_data_dict['x'].append(data_point['x'])
                pitch_data_dict['y'].append(data_point['y'])
    pitch_df = pd.DataFrame(pitch_data_dict)
    return pitch_df

In [ ]:
pitch_df = get_pitch_data_df(flatten_data_list)

Brackuje 90 i 489

In [ ]:
lines = pitch_coords['lines']
lines

In [ ]:
def denormalize_coordinates(x: float, y: float, img_width: int, img_height: int) -> tuple[int, int]:
    return int(x * img_width), int(y * img_height)


def draw_points_on_image(image: np.ndarray, coords: pd.DataFrame) -> np.ndarray:
    image_height, image_width, _ = image.shape

    colors = {
        # 3 punkrtowce
        'Circle central': (0, 255, 255),
        'Circle left': (255, 255, 255),
        'Circle right': (140, 159, 191),
        # 'Goal left crossbar': (255, 128, 128),
        # 'Goal left post left': (128, 255, 128),
        # 'Goal left post right': (128, 128, 255),
        # 'Goal right crossbar': (255, 128, 255),
        # 'Goal right post left': (255, 255, 128),
        # 'Goal right post right': (128, 255, 255),
        # lewe pole karne
        'Big rect. left bottom': (0,0,0),
        'Big rect. left main': (0, 255, 0),
        'Big rect. left top': (255, 0, 0),
        'Small rect. left bottom': (64, 255, 128),
        'Small rect. left main': (192, 255, 32),
        'Small rect. left top': (32, 192, 255),
        #prawe pole karne        
        'Big rect. right bottom': (0, 0, 255),
        'Big rect. right main': (255, 255, 0),
        'Big rect. right top': (255, 0, 255),
        'Small rect. right bottom': (255, 32, 192),
        'Small rect. right main': (32, 255, 192),
        'Small rect. right top': (192, 32, 255),
        #linie
        'Middle line': (255, 64, 128),
        'Side line bottom': (128, 255, 64),
        'Side line left': (64, 128, 255),
        'Side line right': (128, 64, 255),
        'Side line top': (255, 128, 64)
    }
    for i, row in coords.iterrows():
        color = colors[row['line_type']] if row['line_type'] in colors.keys() else (0, 0, 0)
        x_pixel, y_pixel = denormalize_coordinates(row['x'], row['y'], image_width, image_height)
        cv2.circle(image, (x_pixel, y_pixel), radius=5, color=color, thickness=-1)
    # for category, points in coords.items():
    #     color = colors[category]
    #     for point in points:
    #         if category == 'Middle line':
    #             x_pixel, y_pixel = denormalize_coordinates(point['x'], point['y'], image_width, image_height)
    #             cv2.circle(image, (x_pixel, y_pixel), radius=5, color=color, thickness=-1)

    scale_factor = min(1920 / image_width, 1080 / image_height)
    if scale_factor < 1:
        image = cv2.resize(image, (int(image_width * scale_factor), int(image_height * scale_factor)))

    # cv2.imshow("Image with Points", image)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    return image

In [ ]:
sngs = [21, 22, 23, 32, 41, 44, 35]
imgs = [1, 14, 66, 734, 344, 567, 222]
example_images = [(x, y) for x in sngs for y in imgs]
for example_image in example_images:
    image = cv2.imread(
        f'data/SoccerNetGS/gamestate-2024/valid/SNGS-0{example_image[0]}/img1/000{example_image[1]:03d}.jpg')
    transformed_img = draw_points_on_image(image, pitch_df[
        pitch_df['image_id'] == int(f'20{example_image[0]}000{example_image[1]:03d}')])
    cv2.imwrite(f'results\\{example_image[0]}{example_image[1]:03d}.png', transformed_img)

In [ ]:
line_types = pitch_df['line_type'].tolist()
set(line_types)

In [ ]:
#get_YOLO_file('data/SoccerNetGS/train/SNGS-060/', 'train')
#get_YOLO_file('data/SoccerNetGS/test/SNGS-116/', 'test')


In [ ]:
model = YOLO("yolo11n.pt")

results = model("results/21001.png")


# Display the results
results[0].show()

In [ ]:
model = YOLO("yolo11n.pt")
results = model.train(data='datasets/football_data/data.yaml', epochs=30, plots=True)

In [ ]:
dir(results)

In [ ]:
results.task

In [ ]:
model = YOLO("runs/detect/train24/weights/best.pt")
predykted = model.predict(r"C:\Users\Marcin\Videos\ganzgut.mp4", show=True, save=True)

In [ ]:
predykted